In [1]:
import os

def get_file_pairs(exchanges):
    filenames = []
    for directory in os.listdir('data'):
        for filename in os.listdir('data/' + directory):
            if filename.endswith('300.csv'):
                 filenames.append(filename)
    file_pairs = []
    for filename_1 in filenames:
        remaining_filenames = filenames[filenames.index(filename_1)+1:]
        for filename_2 in remaining_filenames:
            for exchange in exchanges:
                if filename_1.replace(exchange, '') in filename_2:
                    file_pairs.append([filename_1, filename_2])
    return file_pairs

exchanges = ['bitfinex', 'coinbase_pro', 'hitbtc']
get_file_pairs(exchanges)

[['bitfinex_eos_usdt_300.csv', 'hitbtc_eos_usdt_300.csv'],
 ['bitfinex_bch_btc_300.csv', 'coinbase_pro_bch_btc_300.csv'],
 ['bitfinex_bch_btc_300.csv', 'hitbtc_bch_btc_300.csv'],
 ['bitfinex_etc_usd_300.csv', 'coinbase_pro_etc_usd_300.csv'],
 ['bitfinex_btc_usd_300.csv', 'coinbase_pro_btc_usd_300.csv'],
 ['bitfinex_ltc_btc_300.csv', 'coinbase_pro_ltc_btc_300.csv'],
 ['bitfinex_ltc_btc_300.csv', 'hitbtc_ltc_btc_300.csv'],
 ['bitfinex_dash_usd_300.csv', 'coinbase_pro_dash_usd_300.csv'],
 ['bitfinex_dash_btc_300.csv', 'coinbase_pro_dash_btc_300.csv'],
 ['bitfinex_dash_btc_300.csv', 'hitbtc_dash_btc_300.csv'],
 ['bitfinex_ltc_usd_300.csv', 'coinbase_pro_ltc_usd_300.csv'],
 ['bitfinex_bch_usdt_300.csv', 'hitbtc_bch_usdt_300.csv'],
 ['bitfinex_bch_usd_300.csv', 'coinbase_pro_bch_usd_300.csv'],
 ['bitfinex_eos_usd_300.csv', 'coinbase_pro_eos_usd_300.csv'],
 ['bitfinex_xrp_usd_300.csv', 'coinbase_pro_xrp_usd_300.csv'],
 ['bitfinex_eth_btc_300.csv', 'coinbase_pro_eth_btc_300.csv'],
 ['bitfinex_

In [2]:
import pandas as pd

def get_df(filename):
    df = pd.read_csv(filename, index_col=0)
    return df

In [3]:
from ta import add_all_ta_features

def engineer_features(df):
    df = add_all_ta_features(df, 'open', 'high', 'low', 'close',
                             'base_volume', fillna=True)
    df = df.drop(columns=['open', 'high', 'low', 'momentum_kama',
                          'momentum_stoch', 'others_cr', 'others_dlr',
                          'trend_ema_fast', 'trend_ema_slow', 
                          'trend_ichimoku_a', 'trend_ichimoku_b', 'trend_kst',
                          'trend_macd', 'trend_visual_ichimoku_a',
                          'trend_visual_ichimoku_b', 'volatility_bbh',
                          'volatility_bbl', 'volatility_bbm',
                          'volatility_dch', 'volatility_dcl',
                          'volatility_kcc', 'volatility_kch',
                          'volatility_kcl'])
    df['time_since_last'] = df['closing_time'].diff(-1)
    return df

In [ ]:
def get_higher_closing_price(df):
    if (df['close_exchange_1'] - df['close_exchange_2']) > 0:
        return 'exchange_1'
    elif (df['close_exchange_1'] - df['close_exchange_2']) < 0:
        return 'exchange_2'
    else:
        return 'equivalent'

def get_pct_higher(df):
    if df['higher_closing_price'] == 'exchange_1':
        return ((df['close_exchange_1'] / 
                 df['close_exchange_2'])-1)*100
    elif df['higher_closing_price'] == 'exchange_2':
        return ((df['close_exchange_2'] / 
                 df['close_exchange_1'])-1)*100
    else:
        return 0
    
def get_target(df):
    if df['pct_higher'] < .55:
        return 'no_arbitrage'
    elif df['higher_closing_price'] == 'exchange_1':
        return 'arbitrage_exchange_2_to_exchange_1'
    elif df['higher_closing_price'] == 'exchange_2':
        return 'arbitrage_exchange_1_to_exchange_2'
        
def merge_dfs(df1, df2):
    df = pd.merge(df1, df2, on='closing_time',
                  suffixes=('_exchange_1', '_exchange_2'))
    df['higher_closing_price'] = df.apply(get_higher_closing_price, axis=1)
    df['pct_higher'] = df.apply(get_pct_higher, axis=1)
    df['target'] = df.apply(get_target, axis=1)
    df = df.drop(columns=['higher_closing_price', 'pct_higher'])
    df['target'] = df['target'].shift(-1)
    df = df[0:-1]
    return df

In [ ]:
def create_all_arbitrage_dfs(exchanges):
    for pair in get_file_pairs(exchanges):
        for exchange in exchanges:
            if exchange in pair[0]:
                exchange_1 = exchange
            if exchange in pair[1]:
                exchange_2 = exchange
        
        df1 = get_df('data/' + exchange_1 + '/' + pair[0])
        df1 = engineer_features(df1)

        df2 = get_df('data/' + exchange_2 + '/' + pair[1])
        df2 = engineer_features(df2)

        df = merge_dfs(df1, df2)
        
        end_of_file_extension = '_' + pair[1].replace('_300', '')
        df.to_csv('arbitrage_data/' + exchange_1 + end_of_file_extension)

exchanges = ['bitfinex', 'coinbase_pro', 'hitbtc']
create_all_arbitrage_dfs(exchanges)

/anaconda3/lib/python3.7/site-packages/ta/trend.py:170: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/anaconda3/lib/python3.7/site-packages/ta/trend.py:174: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
